## OpenAI agents Playground

# 🤖 OpenAI Agents Project

## 🎯 Long-term Goals
- 📚 Learn and understand OpenAI's agent architecture and capabilities
- 🔧 Create custom agents for specific use cases and domains
- 🔄 Integrate agents into other projects and workflows
- 🧩 Experiment with multi-agent systems and orchestration

## 🚀 Features
- Agent-based architecture with specialized roles
- Handoff system for intelligent task routing
- Guardrails for safe and controlled agent behavior
- Async execution for efficient processing

## 💡 Use Cases
- Educational assistants and tutoring
- Research and information gathering
- Workflow automation
- Content generation and curation

## 🛠️ Technologies Used
- OpenAI's agent framework
- Python async programming
- Pydantic for data validation
- Jupyter for interactive development

#### install

In [19]:
# ! pip install openai-agents nest_asyncio


### imports

In [56]:
from pydantic import BaseModel
import dotenv
import nest_asyncio
from agents import Agent, Runner, GuardrailFunctionOutput
from openai.types.responses import ResponseTextDeltaEvent
import asyncio
from typing import AsyncGenerator
from IPython.display import display, Markdown, DisplayHandle

##### Apply nest_asyncio to allow asyncio event loops to be nested

In [2]:
nest_asyncio.apply()

#### Load environment variables

In [3]:
dotenv.load_dotenv()

True

#### First Agent - Math Agent

In [4]:
math_tutor_agent = Agent(
    name="Math Tutor",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

#### Second and Third Agents - History and Science Agents

In [5]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

science_tutor_agent = Agent(
    name="Science Tutor",
    handoff_description="Specialist agent for science questions",
    instructions="You provide help with science problems. Explain concepts clearly and provide examples.",
)

### Handoffs definition

In [6]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, science_tutor_agent, math_tutor_agent]
)

### Agents Orchestration

In [51]:
async def ask_ai(q: str, stream: bool = True) -> AsyncGenerator[str, None]:
    if stream:
        result = Runner.run_streamed(triage_agent, input=q)
        async for event in result.stream_events():
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                yield event.data.delta
    else:
        result = await Runner.run(triage_agent, q)
        yield result.final_output

### Adding Guardrails

In [54]:
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

###### question

In [9]:
question: str = "What is the Pythagorean theorem?"

#### Streaming Response (with Markdown)

In [ ]:
full_response: str = ""
display_id: DisplayHandle = display(Markdown(full_response), display_id=True)

async for chunk in ask_ai(question):
    full_response += chunk
    display_id.update(Markdown(full_response))
    await asyncio.sleep(0.01)

full_response

The Pythagorean theorem is a fundamental principle in geometry that relates to right-angled triangles. It states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. This can be expressed with the formula:

\[ a^2 + b^2 = c^2 \]

where:
- \( c \) is the length of the hypotenuse,
- \( a \) and \( b \) are the lengths of the other two sides.

### Example:

1. **Identifying a Right Triangle:**
   Imagine we have a right triangle with legs of lengths 3 and 4 units.

2. **Applying the Pythagorean Theorem:**
   Plug these values into the formula \( a^2 + b^2 = c^2 \):
   \[
   3^2 + 4^2 = c^2
   \]
   \[
   9 + 16 = c^2
   \]
   \[
   25 = c^2
   \]

3. **Solving for \( c \):**
   To find \( c \), take the square root of both sides:
   \[
   c = \sqrt{25} = 5
   \]

So, the hypotenuse \( c \) is 5 units long.

The Pythagorean theorem is widely used in various applications, including construction, navigation, and physics, to calculate distances and ensure structures are level or squared.

'The Pythagorean theorem is a fundamental principle in geometry that relates to right-angled triangles. It states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. This can be expressed with the formula:\n\n\\[ a^2 + b^2 = c^2 \\]\n\nwhere:\n- \\( c \\) is the length of the hypotenuse,\n- \\( a \\) and \\( b \\) are the lengths of the other two sides.\n\n### Example:\n\n1. **Identifying a Right Triangle:**\n   Imagine we have a right triangle with legs of lengths 3 and 4 units.\n\n2. **Applying the Pythagorean Theorem:**\n   Plug these values into the formula \\( a^2 + b^2 = c^2 \\):\n   \\[\n   3^2 + 4^2 = c^2\n   \\]\n   \\[\n   9 + 16 = c^2\n   \\]\n   \\[\n   25 = c^2\n   \\]\n\n3. **Solving for \\( c \\):**\n   To find \\( c \\), take the square root of both sides:\n   \\[\n   c = \\sqrt{25} = 5\n   \\]\n\nSo, the hypotenuse \\( c \\) is 5 units long.\n\

#### Without Streaming

In [ ]:
full_response = ""
async for chunk in ask_ai(question, stream=False):
    full_response += chunk
display(Markdown(full_response))

The Pythagorean theorem is a fundamental principle in geometry that applies to right triangles. It states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. 

### Formula:
If \( c \) is the length of the hypotenuse and \( a \) and \( b \) are the lengths of the other two sides, the theorem is expressed as:

\[ c^2 = a^2 + b^2 \]

### Example:
Imagine a right triangle where \( a = 3 \) units and \( b = 4 \) units. To find the length of the hypotenuse \( c \):

1. **Square both non-hypotenuse sides:**

   \[
   a^2 = 3^2 = 9
   \]

   \[
   b^2 = 4^2 = 16
   \]

2. **Add these squares:**

   \[
   a^2 + b^2 = 9 + 16 = 25
   \]

3. **Find the square root to determine \( c \):**

   \[
   c = \sqrt{25} = 5
   \]

Therefore, the hypotenuse \( c \) is 5 units long.

### Geometric Significance:
The Pythagorean theorem helps to determine the distance between two points in a plane and is foundational for many areas in mathematics, physics, and engineering, including trigonometry and calculus.

If you have any specific questions or need more examples, feel free to ask!